In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
import matplotlib.pyplot as plt
import gc

2025-01-13 12:35:16.911166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-13 12:35:16.911241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-13 12:35:16.913307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 12:35:16.922716: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# Смешанная точность для экономии GPU памяти
mixed_precision.set_global_policy('mixed_float16')

# Очистка памяти GPU
tf.keras.backend.clear_session()
gc.collect()

1096

In [3]:
# Dataset configurations
patch_size = 256
batch_size = 32
patch_dir = 'dataset_patched_final'

In [4]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)  # Round predictions
    true_positives = K.sum(y_true * y_pred)
    possible_positives = K.sum(y_true)
    predicted_positives = K.sum(y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    return 2 * (precision * recall) / (precision + recall + K.epsilon())

In [5]:
# Define U-Net model
def custom_unet_model(img_height, img_width, img_channels):
    inputs = Input((img_height, img_width, img_channels))
    # Contraction path
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = Dropout(0.3)(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    # Bottleneck
    b = Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    b = Dropout(0.4)(b)
    b = Conv2D(256, (3, 3), activation='relu', padding='same')(b)

    # Expansive path
    u3 = UpSampling2D((2, 2))(b)
    u3 = Concatenate()([u3, c3])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u3)
    c4 = Dropout(0.3)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u2 = UpSampling2D((2, 2))(c4)
    u2 = Concatenate()([u2, c2])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    u1 = UpSampling2D((2, 2))(c5)
    u1 = Concatenate()([u1, c1])
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(u1)
    c6 = Dropout(0.1)(c6)
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(c6)

    outputs = Conv2D(1, (1, 1), activation='sigmoid', dtype='float32')(c6)
    return Model(inputs, outputs)

In [6]:
# Build U-Net
# REQUIREMENT: Model summary must be visible
model = custom_unet_model(patch_size, patch_size, 3)
model.compile(optimizer=Adam(learning_rate=1e-3), 
              loss='binary_crossentropy', 
              metrics=['accuracy', f1])
model.summary()

2025-01-13 12:35:36.525978: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-01-13 12:35:36.657786: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1017 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:25:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 32)         896       ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 256, 256, 32)         0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 256, 256, 32)         9248      ['dropout[0][0]']             
                                                                                              

In [7]:
# Preprocessing functions
def preprocess_image(image):
    return image / 255.0

def preprocess_mask(mask):
    return mask / 255.0

# ImageDataGenerator for data augmentation
train_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image, 
                                         rotation_range=30,
                                         width_shift_range=0.1,
                                         height_shift_range=0.1,
                                         zoom_range=0.2)
train_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

val_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

# Generators
train_image_generator = train_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

train_mask_generator = train_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

val_image_generator = val_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

val_mask_generator = val_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

# Combine generators
def combine_generators(image_gen, mask_gen):
    for (images, masks) in zip(image_gen, mask_gen):
        if masks.ndim == 3:  # Добавляем канал, если его нет
            masks = np.expand_dims(masks, axis=-1)
        yield images, masks

train_generator = combine_generators(train_image_generator, train_mask_generator)
val_generator = combine_generators(val_image_generator, val_mask_generator)

Found 36663 images belonging to 1 classes.
Found 36663 images belonging to 1 classes.
Found 5445 images belonging to 1 classes.
Found 5445 images belonging to 1 classes.


In [8]:
# EarlyStopping()
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

In [9]:
# This is where the magic happens
# REQUIREMENT: The epoch logs must be visible showing validation loss and validation f1.
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_image_generator),
    validation_data=val_generator,
    validation_steps=len(val_image_generator),
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50


2025-01-13 12:35:51.316707: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-01-13 12:35:54.880263: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-01-13 12:35:54.880499: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-01-13 12:35:54.880543: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could

NotFoundError: Graph execution error:

Detected at node model/conv2d_5/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 545, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipykernel

In [ ]:
# REQUIREMENT: Save your model, it is a deliverable
# REQUIREMENT: naming convention 'studentname_studentnumber_unet_model_patchsizepx.h5'
example_file_name = f'artjom_234535_unet_model_{patch_size}px.h5'
print(example_file_name)
model.save(example_file_name)

In [ ]:
# REQUIREMENT: Run and display the output of this cell.
# REQUIREMENT: Do not change the code.
best_val_loss = min(history.history['val_loss'])
best_val_f1 = max(history.history['val_f1'])
print(f"Best validation loss: {best_val_loss}")
print(f"Best validation f1: {best_val_f1}")

In [ ]:
# REQUIREMENT: Run and display the output of this cell.
# REQUIREMENT: Do not change the code.
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = np.arange(1, len(loss) + 1)

plt.plot(epochs, loss, label='train_loss')
plt.plot(epochs, val_loss, label='val_loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss (binary_crossentropy)')
_ = plt.xticks(np.arange(1, len(loss) + 1, 3))

In [ ]:
# REQUIREMENT: Run and display the output of this cell.
# REQUIREMENT: Do not change the code.
train_f1 = history.history['f1']
val_f1 = history.history['val_f1']

epochs = np.arange(1, len(train_f1) + 1)

plt.plot(epochs, train_f1, label='train_f1')
plt.plot(epochs, val_f1, label='val_f1')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('F1')
_ = plt.xticks(np.arange(1, len(loss) + 1, 3))

In [ ]:
### НОВЫЙ ВАРИАНТ с filter_empty_masks

In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import mixed_precision
import gc

# Смешанная точность для экономии GPU памяти
mixed_precision.set_global_policy('mixed_float16')

# Очистка памяти GPU
tf.keras.backend.clear_session()
gc.collect()


# Dataset configurations
patch_size = 256
batch_size = 32
patch_dir = 'dataset_patched_final'
steps_per_epoch = 100
validation_steps = 20

# Check dataset structure
assert os.path.exists(os.path.join(patch_dir, 'train_images')), "Train images directory not found!"
assert os.path.exists(os.path.join(patch_dir, 'train_masks')), "Train masks directory not found!"

# Define F1 metric
def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = TP / (Positives + K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = TP / (Pred_Positives + K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# Define U-Net model
def custom_unet_model(img_height, img_width, img_channels):
    inputs = Input((img_height, img_width, img_channels))
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    c3 = Dropout(0.3)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    b = Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
    b = Dropout(0.4)(b)
    b = Conv2D(128, (3, 3), activation='relu', padding='same')(b)

    u3 = UpSampling2D((2, 2))(b)
    u3 = Concatenate()([u3, c3])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    c4 = Dropout(0.3)(c4)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    u2 = UpSampling2D((2, 2))(c4)
    u2 = Concatenate()([u2, c2])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(c5)

    u1 = UpSampling2D((2, 2))(c5)
    u1 = Concatenate()([u1, c1])
    c6 = Conv2D(16, (3, 3), activation='relu', padding='same')(u1)
    c6 = Dropout(0.1)(c6)
    c6 = Conv2D(16, (3, 3), activation='relu', padding='same')(c6)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c6)
    return Model(inputs, outputs)

# Build U-Net
model = custom_unet_model(patch_size, patch_size, 3)
model.compile(optimizer=Adam(learning_rate=1e-3), 
              loss='binary_crossentropy', 
              metrics=['accuracy', f1])
model.summary()

# Preprocessing functions
def preprocess_image(image):
    return image / 255.0

def preprocess_mask(mask):
    return mask / 255.0

# ImageDataGenerator for data augmentation
train_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image, 
                                         rotation_range=15,
                                         width_shift_range=0.05,
                                         height_shift_range=0.05,
                                         zoom_range=0.1)
train_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

val_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

# Generators
train_image_generator = train_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

train_mask_generator = train_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

val_image_generator = val_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

val_mask_generator = val_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

# Combine generators
def combine_generators(image_gen, mask_gen):
    for (images, masks) in zip(image_gen, mask_gen):
        masks = np.expand_dims(masks, axis=-1) if masks.ndim == 3 else masks
        yield images, masks

train_generator = combine_generators(train_image_generator, train_mask_generator)
val_generator = combine_generators(val_image_generator, val_mask_generator)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

# Training
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    epochs=50,
    callbacks=[early_stopping, reduce_lr],
    workers=4,
    use_multiprocessing=True
)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
### Second try

In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

# Dataset configurations
patch_size = 256
batch_size = 64
patch_dir = 'dataset_patched_final'
steps_per_epoch = 100
validation_steps = 20

def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = TP / (Positives+K.epsilon())
        return recall
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = TP / (Pred_Positives+K.epsilon())
        return precision
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Define U-Net model
def custom_unet_model(img_height, img_width, img_channels):
    inputs = Input((img_height, img_width, img_channels))
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    c3 = Dropout(0.3)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    b = Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
    b = Dropout(0.4)(b)
    b = Conv2D(128, (3, 3), activation='relu', padding='same')(b)

    u3 = UpSampling2D((2, 2))(b)
    u3 = Concatenate()([u3, c3])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    c4 = Dropout(0.3)(c4)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    u2 = UpSampling2D((2, 2))(c4)
    u2 = Concatenate()([u2, c2])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(c5)

    u1 = UpSampling2D((2, 2))(c5)
    u1 = Concatenate()([u1, c1])
    c6 = Conv2D(16, (3, 3), activation='relu', padding='same')(u1)
    c6 = Dropout(0.1)(c6)
    c6 = Conv2D(16, (3, 3), activation='relu', padding='same')(c6)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c6)
    return Model(inputs, outputs)

# Build U-Net
model = custom_unet_model(patch_size, patch_size, 3)
model.compile(optimizer=Adam(learning_rate=1e-3), 
              loss='binary_crossentropy', 
              metrics=['accuracy', f1])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_15 (Conv2D)          (None, 256, 256, 16)         448       ['input_2[0][0]']             
                                                                                                  
 dropout_7 (Dropout)         (None, 256, 256, 16)         0         ['conv2d_15[0][0]']           
                                                                                                  
 conv2d_16 (Conv2D)          (None, 256, 256, 16)         2320      ['dropout_7[0][0]']           
                                                                                            

In [12]:
# Preprocessing functions
def preprocess_image(image):
    return image / 255.0

def preprocess_mask(mask):
    return mask / 255.0

# ImageDataGenerator for data augmentation
train_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image, 
                                         rotation_range=15,
                                         width_shift_range=0.05,
                                         height_shift_range=0.05,
                                         zoom_range=0.1)
train_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

val_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

# Generators
train_image_generator = train_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

train_mask_generator = train_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

val_image_generator = val_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

val_mask_generator = val_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

# Combine generators
def combine_generators(image_gen, mask_gen):
    for (images, masks) in zip(image_gen, mask_gen):
        masks = np.expand_dims(masks, axis=-1) if masks.ndim == 3 else masks
        yield images, masks

train_generator = combine_generators(train_image_generator, train_mask_generator)
val_generator = combine_generators(val_image_generator, val_mask_generator)

Found 36663 images belonging to 1 classes.
Found 36663 images belonging to 1 classes.
Found 5445 images belonging to 1 classes.
Found 5445 images belonging to 1 classes.


In [15]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

# Training
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    epochs=50,
    callbacks=[early_stopping, reduce_lr],
    workers=4,
    use_multiprocessing=True
)

Epoch 1/50


2025-01-13 12:38:39.409827: W tensorflow/core/kernels/conv_ops_gpu.cc:144] None of the algorithms provided by cuDNN frontend heuristics worked; trying fallback algorithms.  Conv: batch: 64
in_depths: 64
out_depths: 64
in: 64
in: 64
filter: 3
filter: 3
filter: 64
dilation: 1
dilation: 1
stride: 1
stride: 1
padding: 1
padding: 1
dtype: DT_HALF
group_count: 1
device_identifier: "sm_8.6 with 51033931776B RAM, 84 cores, 1800000KHz clock, 8001000KHz mem clock, 6291456B L2$"
fusion {
  activation_mode: kRelu
  conv_scale: 1
}
version: 3

2025-01-13 12:38:39.413350: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_fused_impl.h:625 : NOT_FOUND: No algorithm worked!  Error messages:
  Profiling failure on CUDNN engine eng11{}: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 16777216 bytes.
  Profiling failure on CUDNN engine eng0{}: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 16777216 bytes.


NotFoundError: Graph execution error:

Detected at node model_1/conv2d_20/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 545, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipyker

In [ ]:
### First try, corrected for fast training

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.mixed_precision import set_global_policy
import matplotlib.pyplot as plt

# Enable mixed precision
set_global_policy('mixed_float16')

# Dataset configurations
patch_size = 256
batch_size = 16
patch_dir = 'dataset_patched_final'

# Define F1 metric
def f1(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())  # Clip predictions
    y_true = tf.expand_dims(y_true, axis=-1) if y_true.shape.ndims == 3 else y_true
    true_positives = K.sum(y_true * y_pred)
    possible_positives = K.sum(y_true)
    predicted_positives = K.sum(y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    return 2 * (precision * recall) / (precision + recall + K.epsilon())

# Weighted Binary Crossentropy
def weighted_binary_crossentropy(weight):
    def loss(y_true, y_pred):
        y_true = tf.expand_dims(y_true, axis=-1) if y_true.shape.ndims == 3 else y_true
        y_pred = tf.expand_dims(y_pred, axis=-1) if y_pred.shape.ndims == 3 else y_pred
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        weights = tf.where(tf.equal(y_true, 1), weight, 1.0)
        return tf.reduce_mean(bce * weights)
    return loss

weighted_loss = weighted_binary_crossentropy(weight=10.0)

# Define a simplified U-Net model
def custom_unet_model(img_height, img_width, img_channels):
    inputs = Input((img_height, img_width, img_channels))
    # Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    b = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    b = Dropout(0.3)(b)

    # Expansive path
    u2 = UpSampling2D((2, 2))(b)
    u2 = Concatenate()([u2, c2])
    c3 = Conv2D(32, (3, 3), activation='relu', padding='same')(u2)

    u1 = UpSampling2D((2, 2))(c3)
    u1 = Concatenate()([u1, c1])
    c4 = Conv2D(16, (3, 3), activation='relu', padding='same')(u1)

    outputs = Conv2D(1, (1, 1), activation='sigmoid', dtype='float32')(c4)
    return Model(inputs, outputs)

# Build U-Net
model = custom_unet_model(patch_size, patch_size, 3)
model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='binary_crossentropy', 
              metrics=['accuracy', f1])
model.summary()

2025-01-12 18:54:42.962674: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-12 18:54:43.014424: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-12 18:54:43.014476: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-12 18:54:43.014529: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 18:54:43.024909: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-12 18:54:43.025895: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 16)         448       ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 256, 256, 16)         0         ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 128, 128, 16)         0         ['dropout[0][0]']             
 D)                                                                                           

In [2]:
# Preprocessing functions
def preprocess_image(image):
    return image / 255.0

def preprocess_mask(mask):
    return mask / 255.0

# ImageDataGenerator for data augmentation
train_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image, 
                                         rotation_range=30,
                                         width_shift_range=0.1,
                                         height_shift_range=0.1,
                                         zoom_range=0.2)
train_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

val_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

# Generators
train_image_generator = train_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42,
    shuffle=True
)

train_mask_generator = train_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42,
    shuffle=True
)

val_image_generator = val_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42,
    shuffle=False
)

val_mask_generator = val_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42,
    shuffle=False
)

# Combine generators with dimension adjustment for masks
def combine_generators(image_gen, mask_gen):
    for (images, masks) in zip(image_gen, mask_gen):
        if masks.ndim == 3:  # Ensure masks have channel dimension
            masks = np.expand_dims(masks, axis=-1)
        yield images, masks

train_generator = combine_generators(train_image_generator, train_mask_generator)
val_generator = combine_generators(val_image_generator, val_mask_generator)

# Set steps for full dataset training
steps_per_epoch = len(train_image_generator)
validation_steps = len(val_image_generator)

Found 36663 images belonging to 1 classes.
Found 36663 images belonging to 1 classes.
Found 5445 images belonging to 1 classes.
Found 5445 images belonging to 1 classes.


In [3]:
for masks in train_mask_generator:
    print(f"Mask min: {masks.min()}, Mask max: {masks.max()}")
    break

Mask min: 0.0, Mask max: 1.0


In [4]:
# Проверка данных
def check_data(generator):
    for images, masks in generator:
        print(f"Images range: {images.min()} - {images.max()}, shape: {images.shape}")
        print(f"Masks range: {masks.min()} - {masks.max()}, shape: {masks.shape}")
        break

check_data(train_generator)
check_data(val_generator)

Images range: 0.0 - 0.9587311148643494, shape: (16, 256, 256, 3)
Masks range: 0.0 - 1.0, shape: (16, 256, 256, 1)
Images range: 0.0 - 0.7764706015586853, shape: (16, 256, 256, 3)
Masks range: 0.0 - 0.0, shape: (16, 256, 256, 1)


In [5]:
for images, masks in train_generator:
    print(f"Image shape: {images.shape}, dtype: {images.dtype}")
    print(f"Mask shape: {masks.shape}, dtype: {masks.dtype}")
    break

Image shape: (16, 256, 256, 3), dtype: float32
Mask shape: (16, 256, 256, 1), dtype: float32


In [ ]:
# EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    epochs=10,
    callbacks=[early_stopping]
)


Epoch 1/10


In [ ]:
### Second try

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.mixed_precision import set_global_policy
import matplotlib.pyplot as plt

# Enable mixed precision
set_global_policy('mixed_float16')

# Dataset configurations
patch_size = 128  # Smaller patch size for testing
batch_size = 8    # Smaller batch size for faster loading
patch_dir = 'dataset_patched'

# Define F1 score
def f1_smooth(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    true_positives = K.sum(y_true * y_pred)
    possible_positives = K.sum(y_true)
    predicted_positives = K.sum(y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    return 2 * (precision * recall) / (precision + recall + K.epsilon())

# Define U-Net model
def custom_unet_model(img_height, img_width, img_channels):
    inputs = Input((img_height, img_width, img_channels))
    # Contraction path
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = Dropout(0.3)(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    # Bottleneck
    b = Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    b = Dropout(0.4)(b)
    b = Conv2D(256, (3, 3), activation='relu', padding='same')(b)

    # Expansive path
    u3 = UpSampling2D((2, 2))(b)
    u3 = Concatenate()([u3, c3])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u3)
    c4 = Dropout(0.3)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u2 = UpSampling2D((2, 2))(c4)
    u2 = Concatenate()([u2, c2])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    u1 = UpSampling2D((2, 2))(c5)
    u1 = Concatenate()([u1, c1])
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(u1)
    c6 = Dropout(0.1)(c6)
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(c6)

    outputs = Conv2D(1, (1, 1), activation='sigmoid', dtype='float32')(c6)
    return Model(inputs, outputs)

# Build U-Net
model = custom_unet_model(patch_size, patch_size, 3)
model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='binary_crossentropy', 
              metrics=['accuracy', f1_smooth])
model.summary()

2025-01-11 03:02:15.634143: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-11 03:02:15.634200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-11 03:02:15.635616: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-11 03:02:15.643771: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
The dtype policy mixed_float16 may run slowly because

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 32)         896       ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 128, 128, 32)         0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 32)         9248      ['dropout[0][0]']             
                                                                                              

In [2]:
# Preprocessing functions
def preprocess_image(image):
    return image / 255.0

def preprocess_mask(mask):
    return mask / 255.0

# ImageDataGenerator for data augmentation
train_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image, 
                                         rotation_range=20,
                                         width_shift_range=0.1,
                                         height_shift_range=0.1,
                                         zoom_range=0.2)
train_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

val_image_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_mask_datagen = ImageDataGenerator(preprocessing_function=preprocess_mask)

# Generators
train_image_generator = train_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

train_mask_generator = train_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'train_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

val_image_generator = val_image_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_images'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

val_mask_generator = val_mask_datagen.flow_from_directory(
    os.path.join(patch_dir, 'val_masks'),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

# Combine generators
def repeatable_generator(image_gen, mask_gen):
    while True:
        for (images, masks) in zip(image_gen, mask_gen):
            yield images, masks

Found 34672 images belonging to 1 classes.
Found 34672 images belonging to 1 classes.
Found 4928 images belonging to 1 classes.
Found 4928 images belonging to 1 classes.


In [3]:
def partial_generator(image_gen, mask_gen, num_batches):
    count = 0
    while count < num_batches:
        for (images, masks) in zip(image_gen, mask_gen):
            yield images, masks
            count += 1
            if count >= num_batches:
                break

# Toggle between full and partial dataset for training
use_partial_dataset = True
if use_partial_dataset:
    train_batches = 5  # Number of batches for training
    val_batches = 2    # Number of batches for validation
    train_generator = partial_generator(train_image_generator, train_mask_generator, train_batches)
    val_generator = partial_generator(val_image_generator, val_mask_generator, val_batches)
    steps_per_epoch = train_batches
    validation_steps = val_batches
else:
    train_generator = repeatable_generator(train_image_generator, train_mask_generator)
    val_generator = repeatable_generator(val_image_generator, val_mask_generator)
    steps_per_epoch = len(train_image_generator)
    validation_steps = len(val_image_generator)

In [4]:
# EarlyStopping()
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
print("Starting training...")
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    epochs=5 if use_partial_dataset else 50,
    callbacks=[early_stopping],
    verbose=1
)
print("Training completed!")

Starting training...
Epoch 1/5
5/5 [==============================] - 263s 52s/step - loss: 0.6708 - accuracy: 0.7484 - f1_smooth: 6.7487e-05 - val_loss: 0.6233 - val_accuracy: 0.9917 - val_f1_smooth: 6.3079e-05
Epoch 2/5


5/5 [==============================] - 0s 35ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - f1_smooth: 0.0000e+00
Training completed!


In [ ]:
### Third try

In [1]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.mixed_precision import set_global_policy
import matplotlib.pyplot as plt

# Enable mixed precision
set_global_policy('mixed_float16')

# Dataset configurations
patch_size = 128
batch_size = 8
patch_dir = 'dataset_patched'
limited_dir = 'limited_dataset'

# Function to limit dataset size
def limit_dataset(input_dir, output_dir, num_samples):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for subdir in os.listdir(input_dir):
        input_subdir = os.path.join(input_dir, subdir)
        output_subdir = os.path.join(output_dir, subdir)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        files = sorted(os.listdir(input_subdir))[:num_samples]
        for file in files:
            src = os.path.join(input_subdir, file)
            dst = os.path.join(output_subdir, file)
            shutil.copy(src, dst)

# Limit dataset
limit_dataset(os.path.join(patch_dir, "train_images"), os.path.join(limited_dir, "train_images"), num_samples=50)
limit_dataset(os.path.join(patch_dir, "train_masks"), os.path.join(limited_dir, "train_masks"), num_samples=50)
limit_dataset(os.path.join(patch_dir, "val_images"), os.path.join(limited_dir, "val_images"), num_samples=20)
limit_dataset(os.path.join(patch_dir, "val_masks"), os.path.join(limited_dir, "val_masks"), num_samples=20)

# Define F1 score
def f1_smooth(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    true_positives = K.sum(y_true * y_pred)
    possible_positives = K.sum(y_true)
    predicted_positives = K.sum(y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    return 2 * (precision * recall) / (precision + recall + K.epsilon())

# Define U-Net model
def custom_unet_model(img_height, img_width, img_channels):
    inputs = Input((img_height, img_width, img_channels))
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = Dropout(0.3)(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    b = Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    b = Dropout(0.4)(b)
    b = Conv2D(256, (3, 3), activation='relu', padding='same')(b)

    u3 = UpSampling2D((2, 2))(b)
    u3 = Concatenate()([u3, c3])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u3)
    c4 = Dropout(0.3)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u2 = UpSampling2D((2, 2))(c4)
    u2 = Concatenate()([u2, c2])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    u1 = UpSampling2D((2, 2))(c5)
    u1 = Concatenate()([u1, c1])
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(u1)
    c6 = Dropout(0.1)(c6)
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(c6)

    outputs = Conv2D(1, (1, 1), activation='sigmoid', dtype='float32')(c6)
    return Model(inputs, outputs)

# Build and compile model
model = custom_unet_model(patch_size, patch_size, 3)
model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='binary_crossentropy', 
              metrics=['accuracy', f1_smooth])
model.summary()

2025-01-11 02:47:01.780051: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-11 02:47:01.780116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-11 02:47:01.781993: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-11 02:47:01.792353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
The dtype policy mixed_float16 may run slowly because

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 32)         896       ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 128, 128, 32)         0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 32)         9248      ['dropout[0][0]']             
                                                                                              

In [2]:
# ImageDataGenerator
train_image_datagen = ImageDataGenerator(preprocessing_function=lambda x: x / 255.0)
train_mask_datagen = ImageDataGenerator(preprocessing_function=lambda x: x / 255.0)

val_image_datagen = ImageDataGenerator(preprocessing_function=lambda x: x / 255.0)
val_mask_datagen = ImageDataGenerator(preprocessing_function=lambda x: x / 255.0)

# Limited dataset generators
train_image_generator = train_image_datagen.flow_from_directory(
    os.path.join(limited_dir, "train_images"),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

train_mask_generator = train_mask_datagen.flow_from_directory(
    os.path.join(limited_dir, "train_masks"),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

val_image_generator = val_image_datagen.flow_from_directory(
    os.path.join(limited_dir, "val_images"),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='rgb',
    seed=42)

val_mask_generator = val_mask_datagen.flow_from_directory(
    os.path.join(limited_dir, "val_masks"),
    target_size=(patch_size, patch_size),
    batch_size=batch_size,
    class_mode=None,
    color_mode='grayscale',
    seed=42)

# Combine generators
def combine_generators(image_gen, mask_gen):
    for (images, masks) in zip(image_gen, mask_gen):
        yield images, masks

train_generator = combine_generators(train_image_generator, train_mask_generator)
val_generator = combine_generators(val_image_generator, val_mask_generator)

Found 50 images belonging to 1 classes.
Found 50 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.


In [3]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_image_generator),
    validation_data=val_generator,
    validation_steps=len(val_image_generator),
    epochs=10,
    callbacks=[early_stopping]
)

Epoch 1/10
7/7 [==============================] - 338s 47s/step - loss: 0.4718 - accuracy: 0.9906 - f1_smooth: 6.3057e-05 - val_loss: 0.3951 - val_accuracy: 1.0000 - val_f1_smooth: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 333s 47s/step - loss: 0.0520 - accuracy: 0.9914 - f1_smooth: 2.9053e-05 - val_loss: 0.0952 - val_accuracy: 1.0000 - val_f1_smooth: 0.0000e+00
Epoch 3/10
3/7 [===========>..................] - ETA: 3:25 - loss: 0.0055 - accuracy: 0.9920 - f1_smooth: 2.7048e-06

KeyboardInterrupt: 